# ExCAPEDB v1 fix

ExCAPEDB is described in

> Sun J, Jeliazkova N, Chupakin V, Golib-Dzib J-F, Engkvist O, Carlsson L, Wegner J, Ceulemans H, Georgiev I, Jeliazkov V, Kochev N, Ashby TJ, Chen H: ExCAPE-DB: an integrated large scale dataset facilitating Big Data analysis in chemogenomics. J Cheminform 2017, 9:17.  https://jcheminf.biomedcentral.com/articles/10.1186/s13321-017-0203-5

The original release at Zenodo https://zenodo.org/record/173258#.XECnKVUzZlY contains a number of broken SMILES. This notebook:
- Downloads the file from Zenodo
- Check if RDKit is happy with the SMILES
- If not, retrieves the MOL files from the original source (PubChem or ChEMBL)
- Runs standadization with ambit-cli
- Patches the Zenodo file with the new SMILES

In [1]:
import os,path
import datetime
import urllib.request
import logging
from logging.config import fileConfig
from  rdkit  import  Chem
fileConfig('logging.ini')
%run settings.py
%run EXCAPEDB
logger= logging.getLogger()
logger.debug('Started at %s \t%s',os.name, datetime.datetime.now())


2019-01-20 12:13:57,429  DEBUG    Started at nt 	2019-01-20 12:13:57.429606


In [3]:
import pandas as pd

    #print(excapedb.getDB(line))
    #print(excapedb.getIdentifier(line))
    #print(excapedb.getSmiles(line))        
#if os.path.isfile(excapedb[""]) 

excapedb = ExCAPEDB(local_path)
if not os.path.isfile(excapedb.file):
    logger.info("Retrieving file ...")
    urllib.request.urlretrieve(excapedb.zenodo, excapedb.file)
#print(excapedb.file)

## Check SMILES


In [4]:
if not os.path.isfile(excapedb.error_file("pubchem")):
    logger.info("checking smiles ...")
    excapedb.read(excapedb.check_smiles,_max_records=-1)
    #Write identifiers of error SMILES into files (per data source)
    excapedb.write_errors()
else: # already stored , read them back                      
    excapedb.read_errors()

In [5]:
for _db in excapedb.smiles_errors:
    print("{} \tRDkit errors: {}".format(_db,len(excapedb.smiles_errors[_db])))


pubchem 	RDkit errors: 7812
chembl20 	RDkit errors: 7043


## Retrieve from data source

In [6]:
import requests

def retrieve(dbsource,_max_records=-1):
    for db in dbsource:
        try:
            os.mkdir("{}/{}".format(excapedb.path,db))
        except:
            pass
                     
        n=0
        if db == "chembl20":
            #chembl 'SDF' files are neither SDF nor mol ... 
            #they have no $$$$ record tag, but have properties after M END ...
            url="https://www.ebi.ac.uk/chembl/api/data/molecule/{}.sdf"
            file_path="{}/{}/{}.sdf"
            suffix="$$$$"         
        else:
            url="https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{}/record/sdf"
            file_path="{}/{}/{}.sdf"
            suffix=None
        
        for _id in excapedb.smiles_errors[db]:
            file=file_path.format(excapedb.path,db,_id)
            #logger.debug(file)
            if not os.path.isfile(file):
                #urllib.request.urlretrieve(url.format(_id), file)
                try:
                    r = requests.get(url.format(_id))
                    if r.status_code == 200:
                        with open(file, 'wb') as f:  
                            f.write(r.content)
                            if not suffix is None:
                                f.write(suffix.encode())
                    else:
                        logger.error(url.format(_id))
                except Exception as err:
                    logger.error("%s %s",err, url.format(_id))
                    
            n=n+1
            if _max_records>0 and n>_max_records:
                break

excapedb.read_errors()        
dbsource=excapedb.smiles_errors.keys()
retrieve(["pubchem","chembl20"])            

These CHEMBL20 structures do not exists anymore in ChEMBL
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL554792.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL561096.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL1628593.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL561095.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL3210799.sdf
- https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL76164.sdf

The mol files are extracted from MySQL ChEMBL20 dump instead of online query

```
 SELECT chembl_id, concat(molfile,"\n> <chembl_id>\n",chembl_id,"\n\n") FROM molecule_dictionary join  compound_structures using(molregno)
 where chembl_id in ("CHEMBL554792","CHEMBL561096","CHEMBL1628593","CHEMBL561095","CHEMBL3210799","CHEMBL76164")
``` 

## Standardization

- Download ambitcli-4.0.0-20181208.jar from  https://sourceforge.net/projects/ambit/files/Ambit2/AMBIT%20applications/ambitcli/ambitcli-4.0.0/
- Run standardization as in  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5340785/

In [7]:
#Runs ambitcli as external file. Java has to be installed and on the path
import subprocess

tags = {"pubchem" : "-d tag_tokeep=PUBCHEM_CID,PUBCHEM_OPENEYE_ISO_SMILES", "chembl20" : "-d tag_tokeep=chembl_id"}
ambit_bin="{}/ambitcli-4.0.0-20181208.jar".format(excapedb.path)

#for db in ["chembl20","pubchem"]:
for db in ["chembl20"]:
    infile  = "{}/{}/".format(excapedb.path,db);
    outfile_step1 = "{}/{}_step01.txt".format(excapedb.path,db);
    outfile_step2 = "{}/{}_step02.txt".format(excapedb.path,db);
    logfile= "{}/{}.log".format(excapedb.path,db)
    
    if not os.path.isfile(outfile_step1):
        cmp_step1 = "java -jar {} -a standardize -m post -d page=0 -d pagesize=-1 –d tautomers=false {} -d smilescanonical=false –d smiles=true -d inchi=true -i {} -o {}".format(ambit_bin,tags[db],infile,outfile_step1)
        print(cmp_step1)
        subprocess.run(cmp_step1)    

    if not os.path.isfile(outfile_step2):
        cmp_step2 = "java -jar {} -a standardize -m post -d page=0 -d pagesize=-1 –d tag_smiles=AMBIT_SMILES -d tag_inchi=AMBIT_InChI -d tag_inchikey=AMBIT_InChIKey –d tautomers=true -d splitfragments=true -d implicith=true -d smilescanonical=false -d smiles=true -d inchi=true -d neutralise=true -d isotopes=true {} -i {} -o {}".format(ambit_bin,tags[db],outfile_step1,outfile_step2)
        print(cmp_step2)
        subprocess.run(cmp_step2)    

## Patch the distribution

In [8]:
smiles_std={}
ids = {"pubchem" : "PUBCHEM_CID", "chembl20" : "chembl_id"}
for db in ["chembl20","pubchem"]:
    outfile_step2 = "{}/{}_step02.txt".format(excapedb.path,db);
    smiles_std[db] = pd.read_csv(outfile_step2,sep="\t")
    
    display(smiles_std[db].head())

In [ ]:
def process_header(line):
    patched_fh.write("\t".join(line))
    patched_fh.write("\tupdated")
    patched_fh.write("\n")
    
def patch_smiles(num,line,prev_line=None):
    _id = excapedb.getIdentifier(line)
    _db = excapedb.getDB(line)
    _smiles = excapedb.getSmiles(line)
    if (_db=="pubchem"):
        _id=int(_id)
    
    updated=""
    if _id in excapedb.smiles_errors[_db]:
        if num % 100000 == 0:
            logger.info(num)
        #print(_smiles)
        if _db in smiles_std:
            try:
                _tag=ids[_db]
                tmp=smiles_std[_db]
                rows = tmp.loc[tmp[_tag]==_id]
                
                new_smiles=rows['SMILES'].values[0]
                new_inchi=rows['InChI'].values[0]
                new_inchikey=rows['InChIKey'].values[0]
                line = excapedb.replace(line, new_smiles,new_inchikey,new_inchi)
                updated="v2"
            except Exception as err:
                logger.error(err)
                updated="(v2)"
                pass
    #write it        
    patched_fh.write("\t".join(line))
    patched_fh.write("\t")
    patched_fh.write(updated)
    patched_fh.write("\n")
    
            
excapedb.read_errors()        
print( excapedb.smiles_errors["pubchem"])
print( excapedb.smiles_errors["chembl20"])

patched_fh = open(excapedb.file_patched,"w") 
excapedb.read(patch_smiles,_max_records=-1,process_header=process_header)
patched_fh.close()

## Verify the new file

In [10]:
#compress with xz
#and read it
excapedb2=ExCAPEDB(local_path,file_in="pubchem.chembl.dataset4publication_inchi_smiles_v2.tsv.xz",file_patched=None)

#and check the smiles in the new file
excapedb2.read(excapedb2.check_smiles,_max_records=-1,process_header=None)
print(excapedb2.smiles_errors)

{}
['Ambit_InchiKey', 'Original_Entry_ID', 'Entrez_ID', 'Activity_Flag', 'pXC50', 'DB', 'Original_Assay_ID', 'Tax_ID', 'Gene_Symbol', 'Ortholog_Group', 'InChI', 'SMILES', 'updated']


- RDKit still fails to parse 40 ChEMBL SMILES and 127 PubChem SMILES
- No obvious errors in SMILES though

In [11]:
for _db in excapedb2.smiles_errors:
    print("\n{} \tRDkit errors: {}".format(db,len(excapedb2.smiles_errors[_db])))
    tmp=smiles_std[_db]
    _id=ids[_db]
    
    for _value in excapedb2.smiles_errors[_db]:
        if (_db=="pubchem"):
            row = tmp.loc[tmp[_id]==int(_value)]
        else:    
            row = tmp.loc[tmp[_id]==_value]
        smiles=row["SMILES"].values[0]    
        
        entry_id=row[_id].values[0]
        
        print("{}  \t{}".format(entry_id,smiles))
        
        


pubchem 	RDkit errors: 40
CHEMBL458835  	N=1C(=CC=2/C(=C/C3=[NH]C4=C([CH]3)C=C(C(=C4)OC)F)/OC(=O)C2C1S)C
CHEMBL447352  	N=1C(=CC=2/C(=C/C3=[NH]C4=C([CH]3)C=CC(=C4)OC)/OC(=O)C2C1S)C
CHEMBL1342622  	S1C(=C([N+]2=CC=C(C=C2)C3=CC=NC=C3)N=C1O)C=NNC(=O)C=4C([OH2])=[C]C=CC4
CHEMBL501661  	C=1(C=C2C(=CC1)N(CCC(=O)N)C(NC(C=3SC(=CC3)/C=C/C4=NC(=[C]C=C4)[NH3])=O)=N2)N(C)C(C=5C=CC=CC5)=O
CHEMBL211932  	C1=C(C2=CC(OC[C@H](CC=3C4=C(NC3)C=CC=C4)[NH2]C5=[C]C=C6C(=C5)C(=C(N6)O)C(C)=C)=CN=C2)C=C7C(NC(=C7C(C)=C)O)=C1
CHEMBL2047017  	C=1([NH2]C(CCCCCC(O)=NO)=O)C=C(C(=O)NC=2C=C(C(=CC2)F)F)C(=C[C]1)O
CHEMBL458963  	N=1C(C)=CC\2=C(C(=O)O/C2=C\C3=C4C(=CC=C3)[CH]C=[NH]4)C1S
CHEMBL221496  	C1(=CC=C2N=C(C(=CC2=C1)C3=[NH]C4=C(C=C(C=C4)CN5CCCCC5)[CH]3)O)C6=NNC=C6
CHEMBL220902  	C=1(C=C2C=C(C(=NC2=CC1)O)C3=[NH]C4=C(C=C(C=C4)CN5CCCCC5)[CH]3)C6=CC=CC(=[C]6)[OH2]
CHEMBL2047015  	C=1([NH2]C(CCCCCC(O)=NO)=O)C=C(C(=O)NC=2C=CC=CC2)C([OH2])=[C][C]1
CHEMBL183803  	C=12C(=C(O)NC1)C([NH3])=[C]C=C2C3=C[C]=C([NH2]C([NH2]C4=[C]

Find the new version https://zenodo.org/record/173258  (v2)